# 18-5-2-DRILL-Housing-Evaluating performance

## Evaluating performance: House prices model

    - In this exercise, you'll work on your house prices model. To complete this assignment, submit a link to a Jupyter notebook containing your solutions to the following tasks:

    - Load the houseprices data from Thinkful's database.
    - Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

    - Do you think your model is satisfactory? If so, why?

    - In order to improve the goodness of fit of your model, try different model specifications by adding or removing some variables.

    - For each model you try, get the goodness of fit metrics and compare your models with each other. Which model is the best and why?

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
import statsmodels.formula.api as smf
from sqlalchemy import create_engine

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
#postgres_user = 'dsbc_student'
#postgres_pw = '7*.8G9QH21'
#postgres_host = '142.93.121.174'
#postgres_port = '5432'
#postgres_db = 'weatherinszeged'

### Load the data 

In [3]:
#engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    #postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
#weather = pd.read_sql_query('select * from weatherinszeged',con=engine)

# no need for an open connection, as we're only doing a single query
#engine.dispose()


houseprices = pd.read_csv('housing-train.csv')

houseprices.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.000,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.000,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.000,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.000,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.000,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
5,6,50,RL,85.000,14115,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,MnPrv,Shed,700,10,2009,WD,Normal,143000
6,7,20,RL,75.000,10084,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,307000
7,8,60,RL,nan,10382,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,Shed,350,11,2009,WD,Normal,200000
8,9,50,RM,51.000,6120,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2008,WD,Abnorml,129900
9,10,190,RL,50.000,7420,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,1,2008,WD,Normal,118000


In [10]:
houseprices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-n

In [4]:
housePrices_df = houseprices.iloc[:,[18,19,49,50,51,56, 61,80]]

In [5]:
### rename columns
housePrices_df.rename(columns={'OverallCond': 'overallcond', 
                                'YearBuilt': 'yearbuilt', 'FullBath': 'fullbath', 
                                'HalfBath': 'halfbath', 'BedroomAbvGr': 'bedroomabvgr', 'Fireplaces': 'fireplaces',
                                'GarageCars': 'garagecars', 'SaleCondition': 'salecondition', 'SalePrice': 'saleprice'}, inplace=True)

In [7]:
# Y is the target variable
Y = housePrices_df['saleprice'].values

# X is the feature set which includes  
X = housePrices_df[['overallcond', 'yearbuilt', 'fullbath', 'halfbath', 'bedroomabvgr','fireplaces','garagecars']]

In [8]:
### Double check that the model is correct
import statsmodels.api as sm

# We need to manually add a constant
# in statsmodels' sm
X = sm.add_constant(X)

results = sm.OLS(Y, X).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.602
Model:                            OLS   Adj. R-squared:                  0.600
Method:                 Least Squares   F-statistic:                     314.2
Date:                Sat, 15 Feb 2020   Prob (F-statistic):          2.10e-285
Time:                        16:27:41   Log-Likelihood:                -17871.
No. Observations:                1460   AIC:                         3.576e+04
Df Residuals:                    1452   BIC:                         3.580e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.163e+06    1.2e+05     -9.692      0.000    -1.4e+06   -9.28e+05
overallcond   8931.2203   1277.149      6.993      0.000    6425.967    1.14e+04
yearbuilt      581.8616     60.508      9.616      0.000     463.170     700.554
fullbath      3.514e+04   3178.272     11.056      0.000    2.89e+04    4.14e+04
halfbath       1.17e+04   2848.351      4.106      0.000    6109.119    1.73e+04
bedroomabvgr  1965.0399   1878.323      1.046      0.296   -1719.476    5649.556
fireplaces    3.206e+04   2184.994     14.672      0.000    2.78e+04    3.63e+04
garagecars    3.551e+04   2243.336     15.827      0.000    3.11e+04    3.99e+04
==============================================================================
Omnibus:                      712.576   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7410.220
Skew:                           2.021   Prob(JB):                         0.00
Kurtosis:                      13.270   Cond. No.                     1.80e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.8e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Is our model better than an "empty" model?
* When evaluating our model, we first need to ask whether our model contributes anything to the explanation of the outcome variable. 
* In other words, we need to determine whether or not our features explain variance in the outcome. 
* If not, we could drop our features altogether and the resulting "empty" model would perform equally well (which is to say, not very well!).

* we use a F-test for that - in general, F-test represent the ratio between a model's unexplained variance compared to a reduced model. Here, the "reduced model" is a model with no features, meaning all variance in the outcome is unexplained. 

* The F-test null hypothesis states that the model is indistinguishable from the reduced model, which means that the features contribute nothing to the explanation of the target variable.

* if the p-value of the F-test for our model is less than or equal to 0.1 (or even less than or equal to 0.05), we say that our model is useful and contributes something that is statistically significant in the explanation of the target.


###  - Run your house prices model again and assess the goodness of fit of your model using F-test, R-squared, adjusted R-squared, AIC and BIC.

- This model's F statistic is 314.2, and the associated p-value is very close to zero. (Prob (F-statistic):	2.10e-285) which is a very small number and close to 0

This means that we reject the null and means our features add some information to the reduced model and our model is useful in explaining housing prices.

However, F-tests don't quantify how much information our model contributes. This requires R-squared, 

* R-squared is probably the most common measure of goodness of fit in a linear regression model. It is a proportion (between 0 and 1) that expresses how much variance in the outcome variable is explained by the explanatory variables in the model. 

* Generally speaking, higher  𝑅^2 values are better to a point — a low  𝑅^2 indicates that our model isn't explaining much information about the outcome, which means it will not give very good predictions. However, a very high  𝑅^2 is a warning sign of overfitting. 

* A note on negative R-squared values: It is possible to get negative R-squared values for some models. In general terms, if a model is weaker than a straight horizontal line, then R-squared value becomes negative. This usually happens when a constant is not included in the model. Getting a negative value for R-squared means that your model does very poorly in explaining the target.


* In the regression summary table above, we see that the R-squared value of our housing price model is 0.602. This means that our model explains 60.2% of the variance in the charges, leaving 39.8% unexplained. 

# Comparing different models
- in the above example we compared the housing price model with a reduced one. But that's not very interesting so we should create another model and then compare the two.

* Different ways to comapre models
    - Using F-tests
    - Using R-squared
    - Using information criteria
        - two information criteria known as the Akaike Information Criterion (AIC) and 
        Bayesian Information Criterion (BIC).
        - For both AIC and BIC, the lower the value the better. Hence, we choose the model 
        with the lowest AIC or BIC value. Although we can use either of the two criteria, AIC 
        is usually criticized for its tendency to overfit. In contrast, BIC penalizes the 
        number of parameters more severely than AIC and hence favors more parsimonious models 
        (that is, models with fewer parameters).
 
* F-test: model with higher F statistic is superior to the other one
* R-squared: higher R^2 is better
* AIC and BIC: For both AIC and BIC, the lower the value the better

In order to increase the goodness of fit of our model:
* We first log transform the dependent variable. Recall from the checkpoint 2 that the saleprice variable is not normally distributed, so log transforming may help in this regard. To this end, we apply log(1+x) transformation of NumPy.
* Second, we create another variable by summing up the basement, first and the second floor areas.
* Third, we add the interaction of the total area and the overall quality of the house.


In [13]:
housePrices_df.head(10)

,overallcond,yearbuilt,fullbath,halfbath,bedroomabvgr,fireplaces,garagecars,saleprice
0,5,2003,2,1,3,0,2,208500
1,8,1976,2,0,3,1,2,181500
2,5,2001,2,1,3,1,2,223500
3,5,1915,1,0,3,1,3,140000
4,5,2000,2,1,4,1,3,250000
5,5,1993,1,1,1,0,2,143000
6,5,2004,2,0,3,1,2,307000
7,6,1973,2,1,3,2,2,200000
8,5,1931,2,0,2,2,2,129900
9,6,1939,1,0,2,2,1,118000


In [14]:
housePrices_df['total_bath'] = housePrices_df['fullbath'] + housePrices_df['halfbath'] 

In [15]:
housePrices_df.head(10)

,overallcond,yearbuilt,fullbath,halfbath,bedroomabvgr,fireplaces,garagecars,saleprice,total_bath
0,5,2003,2,1,3,0,2,208500,3
1,8,1976,2,0,3,1,2,181500,2
2,5,2001,2,1,3,1,2,223500,3
3,5,1915,1,0,3,1,3,140000,1
4,5,2000,2,1,4,1,3,250000,3
5,5,1993,1,1,1,0,2,143000,2
6,5,2004,2,0,3,1,2,307000,2
7,6,1973,2,1,3,2,2,200000,3
8,5,1931,2,0,2,2,2,129900,2
9,6,1939,1,0,2,2,1,118000,1


In [17]:
# Y is the target variable
Y2 = np.log1p(housePrices_df['saleprice'])
# X is the feature set which includes  
X2 = housePrices_df[['overallcond', 'yearbuilt', 'total_bath', 'bedroomabvgr','fireplaces','garagecars']]

In [18]:
X2 = sm.add_constant(X2)

results = sm.OLS(Y2, X2).fit()

results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.706
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     582.0
Date:                Sat, 15 Feb 2020   Prob (F-statistic):               0.00
Time:                        21:14:00   Log-Likelihood:                 162.70
No. Observations:                1460   AIC:                            -311.4
Df Residuals:                    1453   BIC:                            -274.4
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            1.9516      0.515      3.788      0.000       0.941       2.962
overallcond      0.0708      0.006     12.834      0.000       0.060       0.082
yearbuilt        0.0045      0.000     17.454      0.000       0.004       0.005
total_bath       0.1018      0.010     10.413      0.000       0.083       0.121
bedroomabvgr     0.0448      0.008      5.598      0.000       0.029       0.061
fireplaces       0.1660      0.009     17.586      0.000       0.148       0.185
garagecars       0.1875      0.010     19.524      0.000       0.169       0.206
==============================================================================
Omnibus:                       23.152   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               37.325
Skew:                           0.119   Prob(JB):                     7.85e-09
Kurtosis:                       3.747   Cond. No.                     1.79e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.79e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### * Choose the best one from the three models above with respect to their AIC and BIC scores. Validate your choice by discussing your justification with your mentor

* best model is 
- F-test: model with higher F statistic is superior to the other one
- R-squared: higher R^2 is better
- AIC and BIC: For both AIC and BIC, the lower the value the better

Model 1: 
---------
F-test: 
R-squared:  	0.602
Adj. R-squared:	0.600
AIC:	3.576e+04 = 35760
BIC:	3.580e+04 = 35800


Model 2: 
---------
R-squared:	0.706
Adj. R-squared:	0.705
F-statistic:	582.0
AIC:	-311.4
BIC:	-274.4



AIC and BIC of the model are -311.4 and -311.4 respectively. These values are lower than the ones of the first model. Hence, the second model is better than the first model.